<a href="https://colab.research.google.com/github/ramya1807/Hospital-Playlist/blob/main/Data_Cleaning_Master_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Suzanne

**Legend for the processed data files**



*   master_new.csv
    *   Cleaned master data file 
    *   Each entry: survey respondent and their corresponding mental health indicators, 1 song choice and their corresponding song features
*   data_cleaned.csv
    *   Removed columns that will not be needed for data visualization and modelling (ie. id, song name, artist name)
*   data_cleaned2.csv
    *   data_cleaned.csv file, but with categorical variables (except genres) converted to int8 data type
    *   To be used for modelling

*   data_withgenre.csv
    *   data_cleaned2.csv file, but with the individual genres as columns with binary entries (1s and 0s)











# Preliminary stuff


## Import libraries

In [ ]:
import numpy as np # library for numerical calculations
import pandas as pd # library for reading csv files

## Load data

First, upload the relevant data files to the workspace.

*   master.csv
*   songs_list.csv



In [ ]:
master = pd.read_csv('master.csv') # loading dataset
songs_list = pd.read_csv('songs_list.csv')

# remove NA entries
#master = master.dropna()

master # 235 rows

,timestamp,gender,age,amount_music,life_enjoyment,resilience,balanced_life,emotional_flex,self_actualization,song1,song2,song3,trauma,id,total_health,energy,dance,liveness,valence,tempo,instrumental,acoustic,popularity,health_categorical
0,05/11/2016 14:34:00,Male,18 - 30,1 - 2,4,3,5,3,6,Too Good Drake,Trust Nobody Selena Gomez Tory Lanez Cashmere Cat,Don't Wanna Know Maroon5 Kendrick Lamar,No,0,21,0.633000,0.655000,0.110000,0.571500,98.368000,0.000025,0.281600,71.500000,1.0
1,05/11/2016 14:45:26,Male,18 - 30,1 - 2,6,6,3,5,5,Put that on my set asap rocky ft skepta,High rhe beach lana del rey,New level asap ferg,No,1,25,0.655000,0.526000,0.116000,0.156000,121.127000,0.000000,0.056600,65.000000,1.0
2,05/11/2016 14:45:41,Female,18 - 30,0 - 1,5,5,4,4,5,Fireproof Coleman Hell,Love on the Brain Rhianna,This Girl Kungs vs Cookin' on 3 Burners,No,2,23,0.745500,0.742000,0.183500,0.618500,125.009000,0.000023,0.053800,63.500000,1.0
3,05/11/2016 14:45:55,Male,18 - 30,2+,3,6,5,5,5,Eyes Nose Lips TaeYang,Me Like Yuh Jay Park,Aquaman Jay Park,No,3,24,0.739000,0.681667,0.148900,0.596000,113.576000,0.000001,0.348867,60.333333,1.0
4,05/11/2016 14:49:15,Female,18 - 30,0 - 1,2,5,2,1,5,Sugar Wanderlust,Blue Jeans Lana Del Ray,Mr. Bright side the killers,Yes,4,15,0.539000,0.513000,0.215500,0.291500,122.460500,0.464350,0.599500,3.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
230,08/11/2016 11:48:52,Female,18 - 30,0 - 1,5,5,3,5,5,When I see you again Charlie Puth,GOLD Kiiara,I was here Beyonce,No,230,23,0.519500,0.555000,0.113350,0.321000,112.020500,0.001250,0.523000,69.000000,1.0
231,08/11/2016 12:02:50,Male,18 - 30,1 - 2,2,3,4,3,3,Light up the sky the Afters,Son of man Phil Colins,If Today is your last day Nickelback,Yes,231,15,0.635500,0.520500,0.210300,0.498500,115.990000,0.016350,0.015955,27.000000,0.0
232,08/11/2016 12:38:19,Female,18 - 30,0 - 1,4,5,4,5,5,Reaper Sia,Lost On You Death Valley,Don't Ed Sheeran,Yes,232,23,0.738333,0.686000,0.283033,0.778667,99.346667,0.060037,0.017473,62.000000,1.0
233,08/11/2016 19:39:43,Female,18 - 30,2+,0,3,3,1,1,fake love drake,24K Magic Bruno Mars,daddy lessons beyonce,No,233,8,0.633000,0.800333,0.392467,0.638000,119.664333,0.000005,0.047067,75.333333,0.0


# Format the data

## One song per entry

Convert dataframe to have **one song per entry** and the song metrics/details of the one song (ie. each participant appears 3 times, once for each song, the participants’ gender/age group/mental health details also appear 3 times to account for each song).

In [ ]:
# save versions of the dataframe in which only one of the three song columns remain 
# (no changes are made to all other columns)

# only song 1 entries in dataframe
df_song1 = master.drop(columns=['song2', 'song3'])
df_song1 = df_song1.rename(columns={"song1": "songs"})       # rename song1 column to song
df_song1['song_order'] = 1     # column added to keep track of whether this is song1, song2 or song3
df_song1.head() 

df_song2 = master.drop(columns=['song1', 'song3'])
df_song2.rename(columns={"song2": "songs"})
df_song2 = df_song2.rename(columns={"song2": "songs"})
df_song2['song_order'] = 2     
df_song2.head() 

df_song3 = master.drop(columns=['song1', 'song2'])
df_song3 = df_song3.rename(columns={"song3": "songs"})
df_song3['song_order'] = 3     
df_song3.head() 

,timestamp,gender,age,amount_music,life_enjoyment,resilience,balanced_life,emotional_flex,self_actualization,songs,trauma,id,total_health,energy,dance,liveness,valence,tempo,instrumental,acoustic,popularity,health_categorical,song_order
0,05/11/2016 14:34:00,Male,18 - 30,1 - 2,4,3,5,3,6,Don't Wanna Know Maroon5 Kendrick Lamar,No,0,21,0.6330,0.655000,0.1100,0.5715,98.3680,0.000025,0.281600,71.500000,1.0,3
1,05/11/2016 14:45:26,Male,18 - 30,1 - 2,6,6,3,5,5,New level asap ferg,No,1,25,0.6550,0.526000,0.1160,0.1560,121.1270,0.000000,0.056600,65.000000,1.0,3
2,05/11/2016 14:45:41,Female,18 - 30,0 - 1,5,5,4,4,5,This Girl Kungs vs Cookin' on 3 Burners,No,2,23,0.7455,0.742000,0.1835,0.6185,125.0090,0.000023,0.053800,63.500000,1.0,3
3,05/11/2016 14:45:55,Male,18 - 30,2+,3,6,5,5,5,Aquaman Jay Park,No,3,24,0.7390,0.681667,0.1489,0.5960,113.5760,0.000001,0.348867,60.333333,1.0,3
4,05/11/2016 14:49:15,Female,18 - 30,0 - 1,2,5,2,1,5,Mr. Bright side the killers,Yes,4,15,0.5390,0.513000,0.2155,0.2915,122.4605,0.464350,0.599500,3.000000,0.0,3


In [ ]:
# combine the 3 dataframes (each with only one song per row entry) into a new dataframe (master_new)

master_new = df_song1.append(df_song2, ignore_index=True)
master_new = master_new.append(df_song3, ignore_index=True)
master_new     # 705 rows = 235 x 3

,timestamp,gender,age,amount_music,life_enjoyment,resilience,balanced_life,emotional_flex,self_actualization,songs,trauma,id,total_health,energy,dance,liveness,valence,tempo,instrumental,acoustic,popularity,health_categorical,song_order
0,05/11/2016 14:34:00,Male,18 - 30,1 - 2,4,3,5,3,6,Too Good Drake,No,0,21,0.633000,0.655000,0.110000,0.571500,98.368000,0.000025,0.281600,71.500000,1.0,1
1,05/11/2016 14:45:26,Male,18 - 30,1 - 2,6,6,3,5,5,Put that on my set asap rocky ft skepta,No,1,25,0.655000,0.526000,0.116000,0.156000,121.127000,0.000000,0.056600,65.000000,1.0,1
2,05/11/2016 14:45:41,Female,18 - 30,0 - 1,5,5,4,4,5,Fireproof Coleman Hell,No,2,23,0.745500,0.742000,0.183500,0.618500,125.009000,0.000023,0.053800,63.500000,1.0,1
3,05/11/2016 14:45:55,Male,18 - 30,2+,3,6,5,5,5,Eyes Nose Lips TaeYang,No,3,24,0.739000,0.681667,0.148900,0.596000,113.576000,0.000001,0.348867,60.333333,1.0,1
4,05/11/2016 14:49:15,Female,18 - 30,0 - 1,2,5,2,1,5,Sugar Wanderlust,Yes,4,15,0.539000,0.513000,0.215500,0.291500,122.460500,0.464350,0.599500,3.000000,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
700,08/11/2016 11:48:52,Female,18 - 30,0 - 1,5,5,3,5,5,I was here Beyonce,No,230,23,0.519500,0.555000,0.113350,0.321000,112.020500,0.001250,0.523000,69.000000,1.0,3
701,08/11/2016 12:02:50,Male,18 - 30,1 - 2,2,3,4,3,3,If Today is your last day Nickelback,Yes,231,15,0.635500,0.520500,0.210300,0.498500,115.990000,0.016350,0.015955,27.000000,0.0,3
702,08/11/2016 12:38:19,Female,18 - 30,0 - 1,4,5,4,5,5,Don't Ed Sheeran,Yes,232,23,0.738333,0.686000,0.283033,0.778667,99.346667,0.060037,0.017473,62.000000,1.0,3
703,08/11/2016 19:39:43,Female,18 - 30,2+,0,3,3,1,1,daddy lessons beyonce,No,233,8,0.633000,0.800333,0.392467,0.638000,119.664333,0.000005,0.047067,75.333333,0.0,3


## Sort data 

In [ ]:
# sort the data by id
master_new = master_new.sort_values(['id', 'song_order'])
master_new

,timestamp,gender,age,amount_music,life_enjoyment,resilience,balanced_life,emotional_flex,self_actualization,songs,trauma,id,total_health,energy,dance,liveness,valence,tempo,instrumental,acoustic,popularity,health_categorical,song_order
0,05/11/2016 14:34:00,Male,18 - 30,1 - 2,4,3,5,3,6,Too Good Drake,No,0,21,0.633000,0.655000,0.110000,0.571500,98.368000,0.000025,0.281600,71.500000,1.0,1
235,05/11/2016 14:34:00,Male,18 - 30,1 - 2,4,3,5,3,6,Trust Nobody Selena Gomez Tory Lanez Cashmere Cat,No,0,21,0.633000,0.655000,0.110000,0.571500,98.368000,0.000025,0.281600,71.500000,1.0,2
470,05/11/2016 14:34:00,Male,18 - 30,1 - 2,4,3,5,3,6,Don't Wanna Know Maroon5 Kendrick Lamar,No,0,21,0.633000,0.655000,0.110000,0.571500,98.368000,0.000025,0.281600,71.500000,1.0,3
1,05/11/2016 14:45:26,Male,18 - 30,1 - 2,6,6,3,5,5,Put that on my set asap rocky ft skepta,No,1,25,0.655000,0.526000,0.116000,0.156000,121.127000,0.000000,0.056600,65.000000,1.0,1
236,05/11/2016 14:45:26,Male,18 - 30,1 - 2,6,6,3,5,5,High rhe beach lana del rey,No,1,25,0.655000,0.526000,0.116000,0.156000,121.127000,0.000000,0.056600,65.000000,1.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
468,08/11/2016 19:39:43,Female,18 - 30,2+,0,3,3,1,1,24K Magic Bruno Mars,No,233,8,0.633000,0.800333,0.392467,0.638000,119.664333,0.000005,0.047067,75.333333,0.0,2
703,08/11/2016 19:39:43,Female,18 - 30,2+,0,3,3,1,1,daddy lessons beyonce,No,233,8,0.633000,0.800333,0.392467,0.638000,119.664333,0.000005,0.047067,75.333333,0.0,3
234,12/12/2016 18:59:19,Male,18 - 30,2+,5,5,5,6,6,Crystal Blue Persuasion Tommy James,Yes,234,27,0.521333,0.549000,0.140700,0.591333,112.479333,0.003218,0.236667,53.000000,1.0,1
469,12/12/2016 18:59:19,Male,18 - 30,2+,5,5,5,6,6,You Got It On Justin Timberlake,Yes,234,27,0.521333,0.549000,0.140700,0.591333,112.479333,0.003218,0.236667,53.000000,1.0,2


# Merge data

Merge song features into new master file/dataframe 
*   So song features correspond to individual songs (instead of having them averaged out over the 3 songs each person chooses)

## Prep data for merging

In [ ]:
# remove averaged song features from master_new (will replace these values from the other data file)

to_exclude = ['energy',
              'dance',
              'liveness',
              'valence',
              'tempo',
              'instrumental',
              'acoustic',
              'popularity'
              ]

master_new = master_new.drop(columns=to_exclude) 

master_new

,timestamp,gender,age,amount_music,life_enjoyment,resilience,balanced_life,emotional_flex,self_actualization,songs,trauma,id,total_health,health_categorical,song_order
0,05/11/2016 14:34:00,Male,18 - 30,1 - 2,4,3,5,3,6,Too Good Drake,No,0,21,1.0,1
235,05/11/2016 14:34:00,Male,18 - 30,1 - 2,4,3,5,3,6,Trust Nobody Selena Gomez Tory Lanez Cashmere Cat,No,0,21,1.0,2
470,05/11/2016 14:34:00,Male,18 - 30,1 - 2,4,3,5,3,6,Don't Wanna Know Maroon5 Kendrick Lamar,No,0,21,1.0,3
1,05/11/2016 14:45:26,Male,18 - 30,1 - 2,6,6,3,5,5,Put that on my set asap rocky ft skepta,No,1,25,1.0,1
236,05/11/2016 14:45:26,Male,18 - 30,1 - 2,6,6,3,5,5,High rhe beach lana del rey,No,1,25,1.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
468,08/11/2016 19:39:43,Female,18 - 30,2+,0,3,3,1,1,24K Magic Bruno Mars,No,233,8,0.0,2
703,08/11/2016 19:39:43,Female,18 - 30,2+,0,3,3,1,1,daddy lessons beyonce,No,233,8,0.0,3
234,12/12/2016 18:59:19,Male,18 - 30,2+,5,5,5,6,6,Crystal Blue Persuasion Tommy James,Yes,234,27,1.0,1
469,12/12/2016 18:59:19,Male,18 - 30,2+,5,5,5,6,6,You Got It On Justin Timberlake,Yes,234,27,1.0,2


In [ ]:
# remove unwanted columns from songs_list

to_exclude = ['Unnamed: 0', 
              'Unnamed: 0.1', 
              'Unnamed: 0.1.1'
              ]     
              
songs_list_new = songs_list.drop(columns=to_exclude) 

songs_list_new

,id,songs,song,artist,energy,dance,liveness,valence,tempo,instrumental,acoustic,popularity,genres
0,1,New level asap ferg,New Level,A$AP Ferg,0.655,0.526,0.1160,0.156,121.127,0.000000,0.0566,65.0,"['hip hop', 'pop', 'pop rap', 'rap', 'southern..."
1,3,Eyes Nose Lips TaeYang,"눈,코,입(Eyes, Noes, Lips)",TAEYANG,0.516,0.631,0.2640,0.254,143.777,0.000000,0.7380,67.0,"['k-pop', 'korean r&b']"
2,3,Me Like Yuh Jay Park,Me Like Yuh,Jay Park,0.831,0.715,0.1320,0.805,100.006,0.000004,0.2230,63.0,"['k-hop', 'k-pop', 'korean pop', 'korean r&b']"
3,3,Aquaman Jay Park,Aquaman,Jay Park,0.870,0.699,0.0507,0.729,96.945,0.000000,0.0856,51.0,"['k-hop', 'k-pop', 'korean pop', 'korean r&b']"
4,4,Sugar Wanderlust,Sweet Sugar,Wanderlust,0.813,0.302,0.3680,0.339,126.965,0.082700,0.2550,0.0,"['peruvian indie', 'peruvian rock']"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
541,227,Do it Myself Russ (Drake still trash),Do It Myself,Russ,0.558,0.831,0.0940,0.338,141.974,0.000000,0.0180,60.0,"['hip hop', 'memphis hip hop', 'rap', 'souther..."
542,227,What They Want Russ (TSwift more trash than Dr...,What They Want,Russ,0.404,0.710,0.0953,0.398,139.553,0.000000,0.4840,73.0,"['hip hop', 'memphis hip hop', 'rap', 'souther..."
543,229,Ba Justin beiber,Baby,"Justin Bieber, Ludacris",0.859,0.728,0.1110,0.535,65.043,0.000000,0.0401,79.0,['australian hip hop']
544,230,When I see you again Charlie Puth,See You Again (feat. Charlie Puth),Wiz Khalifa,0.481,0.689,0.0649,0.283,80.025,0.000001,0.3690,81.0,"['pop', 'pop rock']"


## Merge in additional data columns



In [ ]:
# join the dataframes by the songs column

# if we are excluding the id from songs_list, merge with songs_list_new instead

master_new = master_new.join(songs_list_new.set_index('songs'), 
                                  on='songs', 
                                  lsuffix = '_master',
                                  rsuffix = '_song')
master_new

,timestamp,gender,age,amount_music,life_enjoyment,resilience,balanced_life,emotional_flex,self_actualization,songs,trauma,id_master,total_health,health_categorical,song_order,id_song,song,artist,energy,dance,liveness,valence,tempo,instrumental,acoustic,popularity,genres
0,05/11/2016 14:34:00,Male,18 - 30,1 - 2,4,3,5,3,6,Too Good Drake,No,0,21,1.0,1,183.0,Too Good,Drake,0.641,0.820,0.1020,0.395,117.974,0.000051,0.0662,77.0,"['canadian hip hop', 'canadian pop', 'hip hop'..."
235,05/11/2016 14:34:00,Male,18 - 30,1 - 2,4,3,5,3,6,Trust Nobody Selena Gomez Tory Lanez Cashmere Cat,No,0,21,1.0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
470,05/11/2016 14:34:00,Male,18 - 30,1 - 2,4,3,5,3,6,Don't Wanna Know Maroon5 Kendrick Lamar,No,0,21,1.0,3,0.0,Don't Wanna Know,Maroon 5 & Kendrick Lamar,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,05/11/2016 14:45:26,Male,18 - 30,1 - 2,6,6,3,5,5,Put that on my set asap rocky ft skepta,No,1,25,1.0,1,1.0,Put That On My Set (feat. A$AP Rocky & Skepta),ASAP Mob,0.547,0.752,0.1070,0.161,111.962,0.000249,0.1130,52.0,"['hip hop', 'rap', 'southern hip hop', 'trap',..."
236,05/11/2016 14:45:26,Male,18 - 30,1 - 2,6,6,3,5,5,High rhe beach lana del rey,No,1,25,1.0,2,1.0,Lana Del Rey,High by the Beach,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
468,08/11/2016 19:39:43,Female,18 - 30,2+,0,3,3,1,1,24K Magic Bruno Mars,No,233,8,0.0,2,233.0,That's What I Like,Bruno Mars,0.560,0.853,0.0944,0.860,134.066,0.000000,0.0130,86.0,"['pop', 'post-teen pop']"
703,08/11/2016 19:39:43,Female,18 - 30,2+,0,3,3,1,1,daddy lessons beyonce,No,233,8,0.0,3,233.0,Daddy Lessons,Beyoncé,0.851,0.621,0.8870,0.449,90.940,0.000014,0.0202,61.0,"['dance pop', 'pop', 'post-teen pop', 'r&b']"
234,12/12/2016 18:59:19,Male,18 - 30,2+,5,5,5,6,6,Crystal Blue Persuasion Tommy James,Yes,234,27,1.0,1,234.0,Crystal Blue Persuasion,Tommy James & The Shondells,0.313,0.603,0.1840,0.817,99.172,0.000045,0.2480,50.0,"['brill building pop', 'bubblegum pop', 'class..."
469,12/12/2016 18:59:19,Male,18 - 30,2+,5,5,5,6,6,You Got It On Justin Timberlake,Yes,234,27,1.0,2,234.0,You Got It On,Justin Timberlake,0.442,0.574,0.1890,0.467,86.430,0.002900,0.1900,40.0,"['dance pop', 'pop']"


# Data cleaning

## Remove unneeded columns and rows

In [ ]:
# remove other unneeded columns 
# do we want to drop the 'id_master' and 'id_song'?
to_exclude = ['timestamp']

master_new = master_new.drop(columns=to_exclude) 

# remove NA entries (rows with blanks)
# check if we are ok with certain columns having NA entries (eg. song name or artist name?)
# if so, need to code for exceptions to the removal of NA entries
master_new = master_new.dropna(axis=0)  # drop rows with NA entries

master_new   # 551 rows


,gender,age,amount_music,life_enjoyment,resilience,balanced_life,emotional_flex,self_actualization,songs,trauma,id_master,total_health,health_categorical,song_order,id_song,song,artist,energy,dance,liveness,valence,tempo,instrumental,acoustic,popularity,genres
0,Male,18 - 30,1 - 2,4,3,5,3,6,Too Good Drake,No,0,21,1.0,1,183.0,Too Good,Drake,0.641,0.820,0.1020,0.395,117.974,0.000051,0.0662,77.0,"['canadian hip hop', 'canadian pop', 'hip hop'..."
1,Male,18 - 30,1 - 2,6,6,3,5,5,Put that on my set asap rocky ft skepta,No,1,25,1.0,1,1.0,Put That On My Set (feat. A$AP Rocky & Skepta),ASAP Mob,0.547,0.752,0.1070,0.161,111.962,0.000249,0.1130,52.0,"['hip hop', 'rap', 'southern hip hop', 'trap',..."
471,Male,18 - 30,1 - 2,6,6,3,5,5,New level asap ferg,No,1,25,1.0,3,1.0,New Level,A$AP Ferg,0.655,0.526,0.1160,0.156,121.127,0.000000,0.0566,65.0,"['hip hop', 'pop', 'pop rap', 'rap', 'southern..."
2,Female,18 - 30,0 - 1,5,5,4,4,5,Fireproof Coleman Hell,No,2,23,1.0,1,60.0,Fireproof,Coleman Hell,0.774,0.692,0.1410,0.771,128.033,0.000010,0.0149,47.0,"['canadian pop', 'indie poptimism', 'modern al..."
3,Male,18 - 30,2+,3,6,5,5,5,Eyes Nose Lips TaeYang,No,3,24,1.0,1,3.0,"눈,코,입(Eyes, Noes, Lips)",TAEYANG,0.516,0.631,0.2640,0.254,143.777,0.000000,0.7380,67.0,"['k-pop', 'korean r&b']"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
468,Female,18 - 30,2+,0,3,3,1,1,24K Magic Bruno Mars,No,233,8,0.0,2,233.0,That's What I Like,Bruno Mars,0.560,0.853,0.0944,0.860,134.066,0.000000,0.0130,86.0,"['pop', 'post-teen pop']"
703,Female,18 - 30,2+,0,3,3,1,1,daddy lessons beyonce,No,233,8,0.0,3,233.0,Daddy Lessons,Beyoncé,0.851,0.621,0.8870,0.449,90.940,0.000014,0.0202,61.0,"['dance pop', 'pop', 'post-teen pop', 'r&b']"
234,Male,18 - 30,2+,5,5,5,6,6,Crystal Blue Persuasion Tommy James,Yes,234,27,1.0,1,234.0,Crystal Blue Persuasion,Tommy James & The Shondells,0.313,0.603,0.1840,0.817,99.172,0.000045,0.2480,50.0,"['brill building pop', 'bubblegum pop', 'class..."
469,Male,18 - 30,2+,5,5,5,6,6,You Got It On Justin Timberlake,Yes,234,27,1.0,2,234.0,You Got It On,Justin Timberlake,0.442,0.574,0.1890,0.467,86.430,0.002900,0.1900,40.0,"['dance pop', 'pop']"


## Reset index

In [ ]:
# reset the index to tidy the data
master_new = master_new.set_index(np.arange(len(master_new.index)))

master_new

,gender,age,amount_music,life_enjoyment,resilience,balanced_life,emotional_flex,self_actualization,songs,trauma,id_master,total_health,health_categorical,song_order,id_song,song,artist,energy,dance,liveness,valence,tempo,instrumental,acoustic,popularity,genres
0,Male,18 - 30,1 - 2,4,3,5,3,6,Too Good Drake,No,0,21,1.0,1,183.0,Too Good,Drake,0.641,0.820,0.1020,0.395,117.974,0.000051,0.0662,77.0,"['canadian hip hop', 'canadian pop', 'hip hop'..."
1,Male,18 - 30,1 - 2,6,6,3,5,5,Put that on my set asap rocky ft skepta,No,1,25,1.0,1,1.0,Put That On My Set (feat. A$AP Rocky & Skepta),ASAP Mob,0.547,0.752,0.1070,0.161,111.962,0.000249,0.1130,52.0,"['hip hop', 'rap', 'southern hip hop', 'trap',..."
2,Male,18 - 30,1 - 2,6,6,3,5,5,New level asap ferg,No,1,25,1.0,3,1.0,New Level,A$AP Ferg,0.655,0.526,0.1160,0.156,121.127,0.000000,0.0566,65.0,"['hip hop', 'pop', 'pop rap', 'rap', 'southern..."
3,Female,18 - 30,0 - 1,5,5,4,4,5,Fireproof Coleman Hell,No,2,23,1.0,1,60.0,Fireproof,Coleman Hell,0.774,0.692,0.1410,0.771,128.033,0.000010,0.0149,47.0,"['canadian pop', 'indie poptimism', 'modern al..."
4,Male,18 - 30,2+,3,6,5,5,5,Eyes Nose Lips TaeYang,No,3,24,1.0,1,3.0,"눈,코,입(Eyes, Noes, Lips)",TAEYANG,0.516,0.631,0.2640,0.254,143.777,0.000000,0.7380,67.0,"['k-pop', 'korean r&b']"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
546,Female,18 - 30,2+,0,3,3,1,1,24K Magic Bruno Mars,No,233,8,0.0,2,233.0,That's What I Like,Bruno Mars,0.560,0.853,0.0944,0.860,134.066,0.000000,0.0130,86.0,"['pop', 'post-teen pop']"
547,Female,18 - 30,2+,0,3,3,1,1,daddy lessons beyonce,No,233,8,0.0,3,233.0,Daddy Lessons,Beyoncé,0.851,0.621,0.8870,0.449,90.940,0.000014,0.0202,61.0,"['dance pop', 'pop', 'post-teen pop', 'r&b']"
548,Male,18 - 30,2+,5,5,5,6,6,Crystal Blue Persuasion Tommy James,Yes,234,27,1.0,1,234.0,Crystal Blue Persuasion,Tommy James & The Shondells,0.313,0.603,0.1840,0.817,99.172,0.000045,0.2480,50.0,"['brill building pop', 'bubblegum pop', 'class..."
549,Male,18 - 30,2+,5,5,5,6,6,You Got It On Justin Timberlake,Yes,234,27,1.0,2,234.0,You Got It On,Justin Timberlake,0.442,0.574,0.1890,0.467,86.430,0.002900,0.1900,40.0,"['dance pop', 'pop']"


# Export data into new file (master_new.csv)

In [ ]:
master_new.to_csv('master_new.csv', index = False)  

# Prep data for modelling (data_cleaned.csv and data_cleaned2.csv)




In [ ]:
import numpy as np # library for numerical calculations
import pandas as pd # library for reading csv files

In [ ]:
# remove unneeded columns: id, song order number, song names and artist names

master_new = pd.read_csv('master_new.csv')

to_exclude = ['songs',
              'song',
              'artist', 
              'id_master', 
              'id_song',
              'song_order'
              ]     

data_cleaned = master_new.drop(columns=to_exclude)

data_cleaned.to_csv('data_cleaned.csv', index = False)  


In [ ]:
# check data types

data_cleaned.dtypes

gender                 object
age                    object
amount_music           object
life_enjoyment          int64
resilience              int64
balanced_life           int64
emotional_flex          int64
self_actualization      int64
trauma                 object
total_health            int64
health_categorical    float64
energy                float64
dance                 float64
liveness              float64
valence               float64
tempo                 float64
instrumental          float64
acoustic              float64
popularity            float64
genres                 object
dtype: object

In [ ]:
# convert the data type for categorical variables

cat_data = ['gender', 
            'age', 
            'amount_music', 
            'trauma']

# we will not convert the genre column, as we will use textual analysis for the genres

for i in cat_data:
    data_cleaned[i] = pd.Categorical(data_cleaned[i]);
    data_cleaned[i] = data_cleaned[i].cat.codes

data_cleaned.dtypes

gender                   int8
age                      int8
amount_music             int8
life_enjoyment          int64
resilience              int64
balanced_life           int64
emotional_flex          int64
self_actualization      int64
trauma                   int8
total_health            int64
health_categorical    float64
energy                float64
dance                 float64
liveness              float64
valence               float64
tempo                 float64
instrumental          float64
acoustic              float64
popularity            float64
genres                 object
dtype: object

In [ ]:
# save a version of data to be used for data visualization and modelling

data_cleaned.to_csv('data_cleaned2.csv', index = False)  

# Convert genres to columns

## Generate vector of unique genres

In [ ]:
genre_vector = []
for index, row in data_cleaned.iterrows():
  value = str(row['genres'])
  if len(value) != 2 and value != 'nan':
    value = ''.join(value.split()) # remove whitespace
    value = value[1:-1] # remove [ and ]
    values = value.split(",") # split by comma
    i = 0
    while i < len(value.split(",")):
      val = values[i]
      val = val[1:-1]
      genre_vector.append(val)
      i += 1
genre_vector = list(set(genre_vector)) # get unique list of all genres
genre_vector

['progressivetrance',
 'anthemworship',
 'christianmusic',
 'popquebecois',
 'earlymusic',
 'talentshow',
 'miamihiphop',
 'basstrap',
 'redneck',
 'shimmerpop',
 'ccm',
 'electronictrap',
 'deephouse',
 'melodicmetalcore',
 'canadianrock',
 'compositionalambient',
 'pixie',
 'tropical',
 'folkrock',
 'canadianmetal',
 'skatepunk',
 'rockdrums',
 'post-grunge',
 'acappella',
 'italianromanticism',
 'metropopolis',
 'rapmetal',
 'koreanpop',
 'folk-pop',
 'post-rock',
 'bubblegumpop',
 'soul',
 'merseybeat',
 'filmi',
 'christmas',
 'deeptropicalhouse',
 'deepturkishpop',
 'popdance',
 'vaportrap',
 'brillbuildingpop',
 'lgbtq+hiphop',
 'post-teenpop',
 'stomppop',
 'catstep',
 'nigerianhiphop',
 'rapcalme',
 'danishpop',
 'swedishalternativerock',
 'chicagohouse',
 'mellowgold',
 'modernfolkrock',
 'electroswing',
 'torontoindie',
 'downtempo',
 'ninja',
 'neosoul',
 'bhangra',
 'indianfolk',
 'laindie',
 'neo-classical',
 'persianpop',
 'gnawa',
 'electrohouse',
 'dreampop',
 'ukdance

## Create dataframe to store binary values

In [ ]:
genres_col = data_cleaned[['genres']]
genres_col

# 551 rows

,genres
0,"['canadian hip hop', 'canadian pop', 'hip hop'..."
1,"['hip hop', 'rap', 'southern hip hop', 'trap',..."
2,"['hip hop', 'pop', 'pop rap', 'rap', 'southern..."
3,"['canadian pop', 'indie poptimism', 'modern al..."
4,"['k-pop', 'korean r&b']"
...,...
546,"['pop', 'post-teen pop']"
547,"['dance pop', 'pop', 'post-teen pop', 'r&b']"
548,"['brill building pop', 'bubblegum pop', 'class..."
549,"['dance pop', 'pop']"


In [ ]:
# check the number of unique genres
len(genre_vector)

# 387 unique genres (columns)

387

In [ ]:
# Set up dataframe to store binary values of the genres
df = pd.DataFrame(np.zeros((551, 387)), columns=genre_vector)
#df

genres_df = pd.concat([genres_col, df], axis=1)
genres_df

,genres,progressivetrance,anthemworship,christianmusic,popquebecois,earlymusic,talentshow,miamihiphop,basstrap,redneck,shimmerpop,ccm,electronictrap,deephouse,melodicmetalcore,canadianrock,compositionalambient,pixie,tropical,folkrock,canadianmetal,skatepunk,rockdrums,post-grunge,acappella,italianromanticism,metropopolis,rapmetal,koreanpop,folk-pop,post-rock,bubblegumpop,soul,merseybeat,filmi,christmas,deeptropicalhouse,deepturkishpop,popdance,vaportrap,...,technicaldeathmetal,alternativehiphop,newromantic,southernrock,australianhiphop,melodicrap,mississippihiphop,escaperoom,upliftingtrance,undergroundhiphop,skarevival,dancehall,lilith,germanpoprock,strut,stompandholler,protopunk,boyband,shimmerpsych,hindihiphop,britishinvasion,peruvianrock,chicagorap,anti-folk,tracestep,softrock,gangsterrap,gothicmetal,cubaton,norwegianmetal,vaporsoul,alternativer&b,punkblues,chillwave,newwave,garagerock,latinpop,slowcore,britishsoul,trance
0,"['canadian hip hop', 'canadian pop', 'hip hop'...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"['hip hop', 'rap', 'southern hip hop', 'trap',...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"['hip hop', 'pop', 'pop rap', 'rap', 'southern...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,"['canadian pop', 'indie poptimism', 'modern al...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"['k-pop', 'korean r&b']",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
546,"['pop', 'post-teen pop']",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
547,"['dance pop', 'pop', 'post-teen pop', 'r&b']",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
548,"['brill building pop', 'bubblegum pop', 'class...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

## Check for genre and input 1s

In [ ]:
# function to check if the song's genre entry has the specific genre
def has_genre(genre, entry):
  present = False
  #entry = entry.replace(" ", "")
  for i in entry:
    i = str(i)
    i = i.replace(" ", "")
    if i == genre:
      present = True
  return present

In [ ]:
from ast import literal_eval

for row in range(0, genres_df.shape[0]):            # loop through each row
  for col in range(1, genres_df.shape[1]):          # loop through each genre (column) excluding the first column
    entry = literal_eval(genres_df.iat[row, 0])     # convert 'genres' entry to vector
    genre = genres_df.columns.values.tolist()[col]
    genre = genre.replace("'", "")
    genre = str(genre)
    if has_genre(genre, entry):
      genres_df.iat[row, col] = 1

genres_df

,genres,progressivetrance,anthemworship,christianmusic,popquebecois,earlymusic,talentshow,miamihiphop,basstrap,redneck,shimmerpop,ccm,electronictrap,deephouse,melodicmetalcore,canadianrock,compositionalambient,pixie,tropical,folkrock,canadianmetal,skatepunk,rockdrums,post-grunge,acappella,italianromanticism,metropopolis,rapmetal,koreanpop,folk-pop,post-rock,bubblegumpop,soul,merseybeat,filmi,christmas,deeptropicalhouse,deepturkishpop,popdance,vaportrap,...,technicaldeathmetal,alternativehiphop,newromantic,southernrock,australianhiphop,melodicrap,mississippihiphop,escaperoom,upliftingtrance,undergroundhiphop,skarevival,dancehall,lilith,germanpoprock,strut,stompandholler,protopunk,boyband,shimmerpsych,hindihiphop,britishinvasion,peruvianrock,chicagorap,anti-folk,tracestep,softrock,gangsterrap,gothicmetal,cubaton,norwegianmetal,vaporsoul,alternativer&b,punkblues,chillwave,newwave,garagerock,latinpop,slowcore,britishsoul,trance
0,"['canadian hip hop', 'canadian pop', 'hip hop'...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"['hip hop', 'rap', 'southern hip hop', 'trap',...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"['hip hop', 'pop', 'pop rap', 'rap', 'southern...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,"['canadian pop', 'indie poptimism', 'modern al...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"['k-pop', 'korean r&b']",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
546,"['pop', 'post-teen pop']",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
547,"['dance pop', 'pop', 'post-teen pop', 'r&b']",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
548,"['brill building pop', 'bubblegum pop', 'class...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

In [ ]:
genres_df.to_csv('genre.csv', index = False)  

## Add to data for modelling

In [ ]:
# Drop the "genres" column
genres_df = genres_df.drop(columns='genres')

# Combine the master data with the genre columns, for future modeling
data_cleaned3 = pd.concat([data_cleaned, genres_df], axis=1)

data_cleaned3

,gender,age,amount_music,life_enjoyment,resilience,balanced_life,emotional_flex,self_actualization,trauma,total_health,health_categorical,energy,dance,liveness,valence,tempo,instrumental,acoustic,popularity,genres,progressivetrance,anthemworship,christianmusic,popquebecois,earlymusic,talentshow,miamihiphop,basstrap,redneck,shimmerpop,ccm,electronictrap,deephouse,melodicmetalcore,canadianrock,compositionalambient,pixie,tropical,folkrock,canadianmetal,...,technicaldeathmetal,alternativehiphop,newromantic,southernrock,australianhiphop,melodicrap,mississippihiphop,escaperoom,upliftingtrance,undergroundhiphop,skarevival,dancehall,lilith,germanpoprock,strut,stompandholler,protopunk,boyband,shimmerpsych,hindihiphop,britishinvasion,peruvianrock,chicagorap,anti-folk,tracestep,softrock,gangsterrap,gothicmetal,cubaton,norwegianmetal,vaporsoul,alternativer&b,punkblues,chillwave,newwave,garagerock,latinpop,slowcore,britishsoul,trance
0,1,1,1,4,3,5,3,6,0,21,1.0,0.641,0.820,0.1020,0.395,117.974,0.000051,0.0662,77.0,"['canadian hip hop', 'canadian pop', 'hip hop'...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,1,1,6,6,3,5,5,0,25,1.0,0.547,0.752,0.1070,0.161,111.962,0.000249,0.1130,52.0,"['hip hop', 'rap', 'southern hip hop', 'trap',...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,1,1,6,6,3,5,5,0,25,1.0,0.655,0.526,0.1160,0.156,121.127,0.000000,0.0566,65.0,"['hip hop', 'pop', 'pop rap', 'rap', 'southern...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,1,0,5,5,4,4,5,0,23,1.0,0.774,0.692,0.1410,0.771,128.033,0.000010,0.0149,47.0,"['canadian pop', 'indie poptimism', 'modern al...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,1,2,3,6,5,5,5,0,24,1.0,0.516,0.631,0.2640,0.254,143.777,0.000000,0.7380,67.0,"['k-pop', 'korean r&b']",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
546,0,1,2,0,3,3,1,1,0,8,0.0,0.560,0.853,0.0944,0.860,134.066,0.000000,0.0130,86.0,"['pop', 'post-teen pop']",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
547,0,1,2,0,3,3,1,1,0,8,0.0,0.851,0.621,0.8870,0.449,90.940,0.000014,0.0202,61.0,"['dance pop', 'pop', 'post-teen pop', 'r&b']",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
548,1,1,2,5,5,5,6,6,1,27,1.0,0.313,0.603,0.1840,0.817,99.172,0.000045,0.2480,50.0,"['brill building pop', 'bubblegum pop', 'class...",0.0,0.0,0.0,0.0,0.0

## Export data into new file (data_withgenre.csv)

In [ ]:
# Drop the genres column as we will not use it in modeling.
data_withgenre = data_cleaned3.drop(columns='genres')
data_withgenre

,gender,age,amount_music,life_enjoyment,resilience,balanced_life,emotional_flex,self_actualization,trauma,total_health,health_categorical,energy,dance,liveness,valence,tempo,instrumental,acoustic,popularity,progressivetrance,anthemworship,christianmusic,popquebecois,earlymusic,talentshow,miamihiphop,basstrap,redneck,shimmerpop,ccm,electronictrap,deephouse,melodicmetalcore,canadianrock,compositionalambient,pixie,tropical,folkrock,canadianmetal,skatepunk,...,technicaldeathmetal,alternativehiphop,newromantic,southernrock,australianhiphop,melodicrap,mississippihiphop,escaperoom,upliftingtrance,undergroundhiphop,skarevival,dancehall,lilith,germanpoprock,strut,stompandholler,protopunk,boyband,shimmerpsych,hindihiphop,britishinvasion,peruvianrock,chicagorap,anti-folk,tracestep,softrock,gangsterrap,gothicmetal,cubaton,norwegianmetal,vaporsoul,alternativer&b,punkblues,chillwave,newwave,garagerock,latinpop,slowcore,britishsoul,trance
0,1,1,1,4,3,5,3,6,0,21,1.0,0.641,0.820,0.1020,0.395,117.974,0.000051,0.0662,77.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,1,1,6,6,3,5,5,0,25,1.0,0.547,0.752,0.1070,0.161,111.962,0.000249,0.1130,52.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,1,1,6,6,3,5,5,0,25,1.0,0.655,0.526,0.1160,0.156,121.127,0.000000,0.0566,65.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,1,0,5,5,4,4,5,0,23,1.0,0.774,0.692,0.1410,0.771,128.033,0.000010,0.0149,47.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,1,2,3,6,5,5,5,0,24,1.0,0.516,0.631,0.2640,0.254,143.777,0.000000,0.7380,67.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
546,0,1,2,0,3,3,1,1,0,8,0.0,0.560,0.853,0.0944,0.860,134.066,0.000000,0.0130,86.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
547,0,1,2,0,3,3,1,1,0,8,0.0,0.851,0.621,0.8870,0.449,90.940,0.000014,0.0202,61.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
548,1,1,2,5,5,5,6,6,1,27,1.0,0.313,0.603,0.1840,0.817,99.172,0.000045,0.2480,50.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
549,1,1,2,5,5,5,6,6,1,27,1.0,0.442,0.574,0.1890,0.467,86.430,0.002900,0.1900,40.0,0.0,0.0,0.0,0.0,0.

In [ ]:
# Export to csv file
data_withgenre.to_csv('data_withgenre.csv', index = False)  